In [3]:
import pandas as pd
import googlemaps
from datetime import datetime, timezone
from tqdm import tqdm
import csv

tqdm.pandas()

# 1. Define travel time function

Input your own API key for computing travel time (transit, walking) using Google Maps (Distance Matrix) API

In [ ]:
#Uncomment below when you get your own API key from Google Maps API

#API_key = 'Your own API key'
#gmaps = googlemaps.Client(key=API_key)

Define the function for computing travel time between FP and its serving neighborhood (census block group) within 25 miles of the FP

In [2]:
# origin_lat, origin_lon: the latitude/longitude of BG
# address: the address of the BG's nearest food pantry or bank.

def compute_travel_time_bg(origin_lat, origin_lon, address):
    
    origin = '%f,%f' % (float(origin_lat), float(origin_lon))
    destination = address
    
    dt = datetime.strptime("13/02/23 14:30", "%d/%m/%y %H:%M") # change with your time (should be future time and UTC format)   
    dt = dt.replace(tzinfo=timezone.utc)

    result_transit = gmaps.distance_matrix(origin, destination, mode='transit', departure_time=dt)
    result_walk = gmaps.distance_matrix(origin, destination, mode='walking', departure_time=dt)
    
    try:
        if result_transit["rows"][0]["elements"][0]["status"] == "OK":
            if result_walk["rows"][0]["elements"][0]["status"] == "OK":
                duration_transit = result_transit["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_walk = result_walk["rows"][0]["elements"][0]['duration']['value'] / 60
            else:
                duration_transit = result_transit["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_walk = None               
        else:
            if result_walk["rows"][0]["elements"][0]["status"] == "OK":
                duration_walk = result_walk["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_transit = None
            else:
                duration_transit = None
                duration_walk = None
       
        return (duration_transit, duration_walk) # minutes
    
    except IndexError:
        return (None, None)

Example usage of `gmaps.distance_matrix` function

In [7]:
dt = datetime.strptime("13/02/23 11:30", "%d/%m/%y %H:%M")  # change to your time zone in UTC
dt = dt.replace(tzinfo=timezone.utc)

gmaps.distance_matrix('Studio Arts Bldg, Amherst, MA 01003', 'Lederle Graduate Research Tower, Amherst, MA 01002', mode='transit', departure_time=dt)

{'destination_addresses': ['Lederle Graduate Research Tower, Amherst, MA 01002, USA'],
 'origin_addresses': ['456 Sunderland Rd, Amherst, MA 01002, USA'],
 'rows': [{'elements': [{'distance': {'text': '5.1 km', 'value': 5090},
     'duration': {'text': '15 mins', 'value': 917},
     'fare': {'currency': 'USD', 'text': '$1.50', 'value': 1.5},
     'status': 'OK'}]}],
 'status': 'OK'}

# 2. Run `compute_travel_time_bg`

: This will create a csv file `bg_transit_adi.csv` in `data` folder, which contains all information about BGs, their nearset FPs, distance, ADI info, state info, and travel time (both by transit and by walking).

In [4]:
food = pd.read_pickle('../data/fp_bg_pairs.pkl')

`bg_near_pantry` is a dataframe which retrieves each of BGs in `food` dataset and their nearest FPs (≤ 25 miles). 

In [5]:
bg_near_pantry = food.loc[food.groupby('bg_fips')['distance_mi'].idxmin()]
bg_near_pantry = bg_near_pantry[['bg_fips', 'address', 'bg_lat', 'bg_lon', 'distance_mi']].reset_index(drop=True)

In [7]:
bg_near_pantry['address'] = bg_near_pantry['address'].str.strip() # strip white space

In [6]:
bg_near_pantry

,bg_fips,address,bg_lat,bg_lon,distance_mi
0,010010201001,"203 N Court St, Prattville, AL 36067",32.465832,-86.489661,0.835864
1,010010201002,"203 N Court St, Prattville, AL 36067",32.485873,-86.489672,1.684913
2,010010202001,"203 N Court St, Prattville, AL 36067",32.480082,-86.474974,1.065305
3,010010202002,"203 N Court St, Prattville, AL 36067",32.464435,-86.469766,0.327409
4,010010203001,"203 N Court St, Prattville, AL 36067",32.480175,-86.460792,1.367486
...,...,...,...,...,...
238531,560430003012,"620 Big Horn Ave, Worland, WY 82401",44.011329,-107.953427,0.529835
238532,560430003013,"620 Big Horn Ave, Worland, WY 82401",44.018328,-107.955039,0.308140
238533,560430003021,"620 Big Horn Ave, Worland, WY 82401",44.028771,-107.950748,0.966369
238534,560430003022,"620 Big Horn Ave, Worland, WY 82401",44.012966,-107.937466,1.192660


Please be aware that running the function will take significant amount of time (~ 6-7 hours) to complete. 

In [ ]:
bg_near_pantry['transit_time'], bg_near_pantry['walking_time'] = zip(*bg_near_pantry.progress_apply(lambda x:compute_travel_time_bg(x.bg_lat, x.bg_lon, x.address), axis=1))

(OPTIONAL) Save as a csv file that contains travel time between each BG and its nearest pantry

In [9]:
# bg_near_pantry.to_csv('../data/bg_near_pantry_time.csv', index=False) # save as a csv file that contains travel time between each BG and its nearest pantry

## Now, you will get the dataset to conduct accessibility analysis! 

Optional code below as an alternative approach of creating transit time dataset. No need to run. 

In [ ]:
# with open('../data/bg_transit_adi.csv', 'w', encoding='UTF-8') as fd:
    
#     writer = csv.writer(fd)
#     writer.writerow(['bg_fips', 'address', 'distance_mi', 'ADI_NATRANK', 'ADI_STATERNK', 'bg_state', 'address_state', 
#                      'transit_time', 'walking_time'])        
    
#     for i in tqdm(range(len(bg_near_pantry))):
        
#         bg_fips = bg_near_pantry['bg_fips'].iloc[i] # FIPS
#         bg_lat = bg_near_pantry['bg_lat'].iloc[i] # Latitude
#         bg_lon = bg_near_pantry['bg_lon'].iloc[i] # Longitude
#         fp_address = bg_near_pantry['address'].iloc[i] # address
#         bg_distance = bg_near_pantry['distance_mi'].iloc[i] # distance between BG and nearest FP
#         bg_adi = bg_near_pantry['ADI_NATRANK'].iloc[i] # BG's national ADI
#         bg_state_adi = bg_near_pantry['ADI_STATERNK'].iloc[i] # BG's state ADI
#         bg_state = bg_near_pantry['bg_state'].iloc[i] # BG's state
#         fp_state = bg_near_pantry['address_state'].iloc[i] # FP's state
        
#         (transit_t, walk_t) = compute_travel_time_bg(bg_lat, bg_lon, fp_address)
        
#         writer.writerow([bg_fips, fp_address, bg_distance, bg_adi, bg_state_adi, bg_state, fp_state, transit_t, walk_t])

# fd.close()